In [1]:
import subprocess
import os

# Util functions

In [2]:
def biascorrect(input_img, output_img):
    s1=['N4BiasFieldCorrection','--bspline-fitting','[10,4]','-d','3','--input-image']
    s2=['--convergence', '[150x100x50x30,1e-16]', '--output']
    s3=['--shrink-factor', '2']
    
    cmnd=s1+[input_img]+s2+[output_img]+s3
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)
    
def apply_transform(input_img, output_img, ref_img, transformation, segmentation=False):
    # input_img: animal high_res or axial postsurgery img, pre-registered manually to presurgery space
    # output_img: registered high_res or axial postsurgery img to atlas space
    # ref_img: atlas
    
    s1=['antsApplyTransforms', '--default-value', '0', '--float', '0', '--input']
    s2=['--input-image-type', '0', '--interpolation', 'BSpline[5]', '--output']
    s3=['--reference-image']
    s4=['--transform']
    s5=["--interpolation", "GenericLabel"]
    
    cmnd=s1+[input_img]+s2+[output_img]+s3+[ref_img]+s4+[transformation]
    
    if segmentation:
        cmnd = cmnd + s5
        
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)
    
def apply_transform_atlas(input_img, output_img, ref_img, transformation):
    # input_img: atlas
    # output_img: new segmentation specified with animal_id
    # ref_img: unregistered, resampled TurboRARE img of animal_id
    
    s1=['antsApplyTransforms', '--default-value', '0', '--float', '0', '--input']
    s2=['--input-image-type', '0', '--interpolation', 'BSpline[5]', '--output']
    s3=['--reference-image']
    s4=['--transform']
#     s5=["--interpolation", "GenericLabel"]
   
    cmnd=s1+[input_img]+s2+[output_img]+s3+[ref_img]+s4+["["+transformation+", 1]"]
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(cmnd)
    print(result.stderr)
    print(result.stdout)
    return result
    
def apply_transform_segmentation(input_img, output_img, ref_img, transformation):
    # input_img: segmentation file from atlas
    # output_img: new segmentation specified with animal_id
    # ref_img: unregistered, resampled TurboRARE img of animal_id
    # transformation: transformation matrix output of ANTs registration
    
    s1=['antsApplyTransforms', '--default-value', '0', '--float', '0', '--input']
    s2=['--input-image-type', '0', '--interpolation', 'BSpline[5]', '--output']
    s3=['--reference-image']
    s4=['--transform']
    s5=["--interpolation", "GenericLabel"]
   
    cmnd=s1+[input_img]+s2+[output_img]+s3+[ref_img]+s4+["["+transformation+", 1]"]+s5
    result = subprocess.run(cmnd, capture_output=True, text=True)
    print(result.stdout)


In [3]:
root="/Users/eminhanozil/Dropbox (Yanik Lab)/Localization Manuscript 2024/RAT DATA"

dwi_path = os.path.join(root, "WHS_SD_rat_atlas_v4_pack","WHS_SD_rat_DWI_v1.01.nii.gz")
mask_path = os.path.join(root, "WHS_SD_rat_atlas_v4_pack","WHS_SD_v2_brainmask_bin.nii.gz")
t2s_path = os.path.join(root, "WHS_SD_rat_atlas_v4_pack","WHS_SD_rat_T2star_v1.01.nii.gz")
atlas_path = os.path.join(root, "WHS_SD_rat_atlas_v4_pack","WHS_SD_rat_atlas_v4.nii.gz")

In [4]:
animal_id="rEO_10"
session="postsurgery20241024"
bids_work="_ind_type_40"

In [9]:
filename = "duo-relaxation_heatmap-sagittal-right_bundle-aligned_to_alignedSagittal.nii.gz"
segmentation = False

session="postsurgery20241024"

# session="postsurgery20240924"
input_path = os.path.join(root, animal_id, "mri", "ses-"+session, "anat", filename)

transformation = os.path.join(root, animal_id, "mri", "ses-"+session, bids_work, "s_register", "output_Composite.h5")

output_filename = "duo-relaxation_heatmap-sagittal-right_bundle-aligned_to_alignedSagittal-registered.nii.gz"
output_path = os.path.join(root, animal_id, "mri", "ses-"+session, "anat", output_filename)

result = apply_transform(input_path, output_path, t2s_path, transformation, segmentation)

In [ ]:
input_img="/Users/eminhanozil/electrode-localization/mri/atlas/WHS_SD_rat_DWI_v1.01.nii.gz"
transformation="/Users/eminhanozil/electrode-localization/mri/rEO_05/output_Composite.h5"
ref_img="/Users/eminhanozil/electrode-localization/mri/rEO_05/sub-rEO_05_ses-presurgery_task-rest_acq-TurboRARE_T2w-ind_10_resampled.nii.gz"
output_img="/Users/eminhanozil/electrode-localization/mri/rEO_05/rEO_05-elastic-dwi.nii.gz"

result=apply_transform_atlas(input_img, output_img, ref_img, transformation)